In [5]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from farmsize import data_prep, db_scan, mapping
from shapely.geometry import Point


In [12]:
country_mapping = data_prep.load_json("./data/country_mappings.json")

# Loading the administrative data
ipums_level1 = gpd.read_file('./data/mapping/ipums/world_geolev1_2020/world_geolev1_2020.shp')
ipums_level2 = gpd.read_file('./data/mapping/ipums/world_geolev2_2020/world_geolev2_2020.shp')

# Subsetting it to match the countries 
ipums_level1 = ipums_level1.loc[ipums_level1["CNTRY_NAME"].isin(country_mapping["ipums"]),:]
ipums_level2 = ipums_level2.loc[ipums_level2["CNTRY_NAME"].isin(country_mapping["ipums"]),:]

# Adding ISO country details
ipums_level1 = ipums_level1.merge(country_mapping, left_on="CNTRY_NAME", right_on="ipums", how="left")
ipums_level2 = ipums_level2.merge(country_mapping, left_on="CNTRY_NAME", right_on="ipums", how="left")

# Loading RHoMIS Indicators
indicator_data = pd.read_csv("./data/RHoMIS_Indicators.csv", encoding="latin")
countries_iso_2 = country_mapping["iso_2"].to_list()
indicator_data = data_prep.subset_data(indicator_data, complete_gps=True, countries=countries_iso_2) # Subsetting data bas

In [ ]:
# World Shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
# Generated using the function adminstrative_boundaries = mapping.get_admin_boundaries_multiple_countries(...)
admin_1 = mapping.read_admin_boundaries_file('./data/mapping/subnational_geometries.csv')
admin_1

In [ ]:
# Spatially joining the two datasets
indicator_data["geometry"] = [Point(xy) for xy in zip(indicator_data["GPS_LON"], indicator_data["GPS_LAT"])]
geo_indicator = gpd.GeoDataFrame(indicator_data)
geo_indicator = geo_indicator.sjoin(world, how="left", op="within")


In [ ]:
# Plot the points on a map
#same as
#fig= plt.figure()
#ax=fig.add_subplot()
fig, ax =  plt.subplots(figsize=(15,15))
ax.set_aspect('equal')
#Plot map layer
world.loc[world["continent"]=="Africa",].plot(ax=ax,color="white", edgecolor="black")

world.loc[world["iso_a3"].isin(countries_iso_3),].plot(ax=ax,color="blue", edgecolor="black", alpha=0.1)

#Plot Points
geo_indicator.plot(ax=ax, marker=0, color='black', markersize=5)

fig.show()
fig.savefig('./outputs/exploratory/map.png')


In [ ]:
# Grouping by 
row_subsets = indicator_data["LandCultivated"].notna() & indicator_data["LandCultivated"].between(0.05,100)
column_subsets= ["ID_COUNTRY", "LandCultivated"]
grouping="ID_COUNTRY"

fig, ax = plt.subplots(figsize=(10,10))
plt.tight_layout()
ax.set_title("KDE of Land ")
ax.set_xlabel("Land Cultivated (ha)")
ax.set_ylabel("Density")
ax.set_xlim([0,25])
ax.set_ylim([0,0.6])
indicator_data.loc[row_subsets,:].groupby("ID_COUNTRY")["LandCultivated"].plot(kind="kde", ax=ax, legend=True)
fig.savefig("./outputs/exploratory/land_size_kde_all_countries.png", bbox_inches="tight")

# Clustering Households Spatially

In [ ]:
cluster_labels = db_scan.cluster_gps_points(indicator_data, "GPS_LON", "GPS_LAT", epsilon=0.1)